
# TODO:
- Pull data from other sources (retirement/dividend etc...)
- Fix the fucking pulled from saving (both ways are triggered, it is fine except monthly sum migh be negative enough to show up)


In [ ]:
# Installation of requirements, it really depend on your environment
## Kinda very needed
# !pip instal python_dateutil # or install based on you your env 
# !conda install -c conda-forge numpy # installed with conda
# !conda install -c conda-forge pandas #  installed with conda 

## Make pretty graph notebook
# !conda install -c conda-forge matplotlib  
# !conda install -c plotly plotly 



In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go 
import plotly.express as px 
import pprint

## This contains the actual classification mapping
import dictdata 

### These might need to be tuned to match display
plt.rcParams['figure.figsize'] = [150, 150]
plt.rcParams['figure.dpi'] = 50
mpl.rcParams['font.size'] = 200.0
mpl.rcParams['font.size'] = 200.0

# Global variables And settings
YEAR_IN_REVIEW=2021

pp = pprint.PrettyPrinter(indent=4)
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', 1000)


In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Helper function(s)
def clean_currency(x):
    """ If the value is a string, then remove wrong delimeter. Ideally this would be leveraging LC_NUMERIC
    Or if very lazy or scramble data something like https://github.com/hayj/SystemTools/blob/master/systemtools/number.py
    """
    if isinstance(x, str):
        return(x.replace(',', '.'))
    return(x)

# Not used, by this project at least
# def parseAmount(amountString):
#     try:
#         return float(str(amountString.replace(",", ".")))
#     except ValueError as ve:
#         print(ve)
#         pass

In [ ]:
# XXX Ideally a way to upload the csv directly via a widget and/or UI
path="/home/lfx/Download/2021.tab"

In [ ]:
# This is where you match whatever format you have in the file and give it the following header
df = pd.read_csv(path,delimiter='\t',header=None,names=['Account Number','Currency','Date','Amount before transaction','Amount after transaction','Date of transaction','Transaction','Note'])
df['Transaction'] = df['Transaction'].apply(clean_currency).astype('float')

In [ ]:
# Show a 
df.head()

In [ ]:
# More global data
full_value = 0

# we want a dict that will hold our results of aggregate values by catergory
value_by_category = {}

# for coolness reason we also track the spend on each shop
value_by_shop = {}

# we want a dict that will hold our spent per card
value_by_card = {}

# and here we will track all the lines that we could not match to a shop
unknown_shops = []

In [ ]:
## This is probably wrong and ugly, iterrows iterate over a series but not the original df ?
## Also the repeated usage of "row[X]" is unsettling 
for index, row in df.iterrows():
    amount = row['Transaction']
    # if we caught an exception parsing, then we won't have an amount here
    # instead it will be `None`, so ignore it and move on to the next item in the loop
    if amount is None:
        continue
    if amount <0:
        df.at[index,'Out']=abs(amount)
        df.at[index,'In']=0.0
    else:
        df.at[index,'Out']=0.0
        df.at[index,'In']=amount

    # ok this is shit and nasty it took me some time to realize that you don't have the shop name cleanly
    # and it's just a comment field with everything.
    # Ok now we want to optimize the loop. What you are doing in your if/elif block is:
    #   You take the item from row[7]
    #   Loop over each list in a hardcoded way
    #   If you match you add it to a hardcoded aggregate value per category.

    # First we take the string from row[7]
    shop_string = row['Note']
    found = False
    ## Add field "Date Of Execution" date string with actual date because replacing is hard and i am lazy
    date_time_obj = datetime.strptime(str(row['Date of transaction']), '%Y%m%d')
    df.at[index,'Date Of Execution']=date_time_obj
    for category, shops in list(dictdata.shops_in_category.items()):
        #print(dictdata.shops_in_category.items())
        for shop in shops:
            if shop in shop_string:
                found = True
                #print("shop: " + shop +" category  "+ category)
                df.at[index,'tags']=category
                # Now if the shop is in this category we want to add the amount spent to the aggregate of amounts spent for this category:
                if category in value_by_category:
                    value_by_category[category] += amount
                else:
                    value_by_category[category] = amount

                # We also add it to the total spend per shop:
                if shop in value_by_shop:
                    value_by_shop[shop] += amount
                else:
                    value_by_shop[shop] = amount
                    # If we don't want a shop to belong to multiple categories, then as soon as we match it [this `if` statement executes] we want to stop looping, we can do that by using a `break` statement:
                    # If you want to have the same shop in multiple categories, then just remove this `break` statement.
                break
        # Let's keep track of all the ones we didn't find so we can categorize them
    if not found:
        unknown_shops.append(shop_string)
        df.at[index,'tags']= "UNKNOWN"
        #print(shop_string)

        # Now we also want to add the amount to the total value:
    full_value += amount
    

    paid_by_card = False
    ## While we are looping lets get spend per card holder, this rely on PASXXX in the TSV file
    for owner,cards in list(dictdata.card_owner.items()):
        for card in cards:
            if card in shop_string:
                paid_by_card = True
                df.at[index,'Card Holder']=owner
                # Now if the shop is in this category we want to add the amount spent to the aggregate of amounts spent for this category:
                if card in value_by_card:
                    value_by_card[card] += amount
                else:
                    value_by_card[card] = amount

    
    if not paid_by_card:
        df.at[index,'Card Holder']="automatic"
        if "automatic" in value_by_card:
            value_by_card["automatic"] += amount
        else:
            value_by_card["automatic"] = amount

## ITs this to iterate a second time but i would rather not fuck it up, again            
for index, row in df.iterrows():
    tags = row['tags']
    budget_type_found = False
    for budget_type,tag_value  in list(dictdata.category_in_Budget.items()):
        for tag in tag_value:
            if tags in tag:
                budget_type_found = True
                df.at[index,'Budget Type']=budget_type
        if not budget_type_found:
            df.at[index,'Budget Type']="Wants"


In [ ]:
df.head()

### This is the part of the feedback loop if your dictdata.py is empty
Running this cell will return a list of "unknown" shops. Usually you would have a look and pick the largest/unique string that qualify the most shop of the same type and update "dictdata.py"

In [ ]:
#pp.pprint('full_value %s' % (full_value))
#pp.pprint(value_by_category)
#pp.pprint(value_by_shop)
pp.pprint('%d unknown shops' % len(unknown_shops))
for unknown in unknown_shops:
    print(unknown)

In [ ]:
# XXX Not displayed but shops in subgroups are also collected 
# This is ugly and old in the current iter, we have In/Out fields and shouldnt rely on abs()
value_by_shop_list = []
for k in value_by_shop.items():
    val=(k[0],abs(k[1]))
    value_by_shop_list.append(val)

df_value_by_shop = pd.DataFrame.from_records(
    value_by_shop_list, columns=['Shop', 'Total Spent'])

df_value_by_shop.loc[df_value_by_shop['Total Spent'] < 500, 'Shop'] = 'Other vendor' # Represent only large shops
fig = px.pie(df_value_by_shop, values='Total Spent', names='Shop', title='Euro Value per vendor')

fig.show()


In [ ]:
# Save all income in a single CSV file
Income = df.loc[df['tags'].isin(['Salary','Dividends','Refunds'])]
Income.to_csv(r'Incomedata.csv')
# Save all the data back with all the categories 
df.to_csv(r'Fulldata.csv')

In [ ]:
# Grouping: https://towardsdatascience.com/how-to-group-data-by-different-time-intervals-using-python-pandas-eb7134f9b9b0
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

#grouped = df.groupby([pd.Grouper(key='Date Of Execution', freq='M'), 'tags']).Transaction.sum().plot()

## Result is a series, using to frame to just not bother 
grouped = df.groupby([pd.Grouper(key='Date Of Execution', freq='M'), 'tags','Budget Type','Card Holder']).Transaction.sum().to_frame()

for index,row in grouped.iterrows():
    amount = row['Transaction']
    if amount <0:
        grouped.at[index,'Out']=abs(amount)
        grouped.at[index,'In']=0.0
    if amount > 0:
        grouped.at[index,'Out']=0.0
        grouped.at[index,'In']=amount

#print(grouped.sort_values(by=['Date Of Execution'])) # No longer needed as it is sorted by default 
# Saved grouped to CSV
grouped.to_csv(r'grouped.csv')



In [ ]:
### XXX This is ugly as sin and probably the worse way to do this
### XXX Column name is wrong, it should be period and it should be just "YYYY-MM"
### Instead i am being lazy and picking "YYYY-MM-last-day"

### In the mean time we iterate over the dates but we pull data from the file we saved earlier
# XXX df_grouped is used heavily because due to lazyness the group_by  in the above cell 
#     create a different index (tag is no long a column by itself)
dataframes_per_month = {}
df_grouped = pd.read_csv('grouped.csv')

for month in range(1,13):
     ## Worse way to get the last day of the year
     monthly=date(YEAR_IN_REVIEW,month,1)
     monthly=monthly + relativedelta(day=31)
     
     #print(monthly)
     df3_monthly_temp = df_grouped.loc[df_grouped['Date Of Execution'] == monthly.strftime('%Y-%m-%d')].sort_values(by='Transaction')
     dataframes_per_month[monthly]=df3_monthly_temp

     titleofgraph = ("Break down for "+str(monthly)[:-3])

    
     fig=px.sunburst(dataframes_per_month[monthly],values='Out',path=["Budget Type","tags","Card Holder"],
                     width=800,title = titleofgraph)
     fig.update_traces(textinfo='label+percent root+value')
     fig.show()
     #fig.write_image("output/"+str(month).zfill(2)+".png", engine="kaleido")

In [ ]:
df_grouped.head()

In [ ]:
# At this point i am just trying around different things.

In [ ]:
Yearly_category_sums = abs(grouped.groupby("tags").sum())
ax = Yearly_category_sums.plot.pie(y="Transaction")
ax.legend(loc="upper left", bbox_to_anchor=(1.5, 1))

In [ ]:
Yearly_category_sums.sort_values(by=['Transaction'])

In [ ]:
# XXX DEFO does not work with lots of smaller percentage, the display is bad.
df_yearly_breakdown = df_grouped.copy(deep=True)
df_yearly_breakdown.loc[df_yearly_breakdown['Out'] < 200, 'tags'] = 'Other shops' # Represent only large purchases/transactions
fig = px.pie(
     df_grouped, values = 'Out',
        names = 'tags', title = 'Break down of the full year',
        color_discrete_sequence = px.colors.sequential.Magma )
fig.show()


In [ ]:
df_yearly_breakdown.head()